In [1]:
import keras
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

Using TensorFlow backend.


In [2]:
##  補註解


In [13]:
##  圖片的維度
sizeImageInput       = (32,32,3)  

##  攤平後要加入的變數維度
sizeDescriptionInput = (3,)  

imageInput       = keras.Input(sizeImageInput, name='ImageInput')

##  第一組卷積層
modelStructure   = keras.layers.Convolution2D(filters=32, kernel_size=(3,3), strides=1, padding="same", 
                                              activation="relu", name="Conv-1")(imageInput)
modelStructure   = keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name = "MaxPooling-1")(modelStructure)

'''
##  第二組卷積層
modelStructure   = keras.layers.Convolution2D(filters=32, kernel_size=(3,3), strides=1, padding="same", 
                                              activation="relu", name="Conv-2")(modelStructure)
modelStructure   = keras.layers.MaxPooling2D(pool_size=(2, 2), strides=None, name="MaxPooling-2")(modelStructure)
'''

##  攤平
modelStructure   = keras.layers.Flatten(name="Flat")(modelStructure)

##  加入新變數
descriptionInput = keras.Input(sizeDescriptionInput, name="DescriptionInput")
modelStructure   = keras.layers.concatenate([modelStructure, descriptionInput], name="Merge")

##  隱藏層
modelStructure   = keras.layers.Dense(200, activation="tanh", name="Hidden-1")(modelStructure)

'''
##  隱藏層
modelStructure   = keras.layers.Dense(200, activation="tanh", name="Hidden-2")(modelStructure)


##  隱藏層
modelStructure   = keras.layers.Dense(50, activation="tanh", name="Hidden-3")(modelStructure)
'''

##  輸出層
modelStructure   = keras.layers.Dense(2, activation="softmax", name="Output")(modelStructure)

##  建構模型框架
model = keras.Model([imageInput, descriptionInput], modelStructure)



##  設定學習參數
model.compile(optimizer=keras.optimizers.Adadelta(lr=0.1, rho=0.95, epsilon=None, decay=0.0), 
              loss='categorical_crossentropy', metrics=['accuracy'])



print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ImageInput (InputLayer)         (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
Conv-1 (Conv2D)                 (None, 32, 32, 32)   896         ImageInput[0][0]                 
__________________________________________________________________________________________________
MaxPooling-1 (MaxPooling2D)     (None, 16, 16, 32)   0           Conv-1[0][0]                     
__________________________________________________________________________________________________
Flat (Flatten)                  (None, 8192)         0           MaxPooling-1[0][0]               
__________________________________________________________________________________________________
Descriptio

In [14]:
##  測試是否可以執行模型

path      = "C:\\Users\\user\\Desktop\\Python\\projectNevus\\data\\temporary\\"
file      = "dataInputFull.txt"
dataInput = pickle.load(open(path + file, "rb"))

Index    = dataInput[0]
Id       = dataInput[1]
Image    = dataInput[2] / 255  ## 正規化(不好的寫法)
Variable = dataInput[3]
Label    = keras.utils.to_categorical(dataInput[4])

IndexTrain, IndexValid, LabelTrain, LabelValid = train_test_split(Index, Label, stratify=Label, test_size=0.1)

##  設定 Early stopping
EarlyStopping = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='min')

model.fit(x=[Image[IndexTrain, :, :, :], Variable[IndexTrain, :]], y=LabelTrain, 
          batch_size=256, epochs=12, callbacks=[EarlyStopping],
          validation_data = ([Image[IndexValid, :, :, :], Variable[IndexValid, :]], LabelValid))


Train on 18002 samples, validate on 2001 samples
Epoch 1/12
18002/18002 [==============================] - 10s 554us/step - loss: 0.2400 - acc: 0.9032 - val_loss: 0.2153 - val_acc: 0.9035
Epoch 2/12
18002/18002 [==============================] - 10s 545us/step - loss: 0.2184 - acc: 0.9043 - val_loss: 0.2353 - val_acc: 0.9005


In [15]:
confusion_matrix(model.predict([Image[IndexValid, :, :, :], Variable[IndexValid, :]]).argmax(axis=-1), LabelValid.argmax(axis=-1))

array([[   1,    1],
       [ 198, 1801]], dtype=int64)